In [1]:
import pandas as pd
import sqlite3

In [2]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB


In [4]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB


In [6]:
conn = sqlite3.connect("restaurants.db")

with open("restaurants.sql", "r") as file:
    sql_script = file.read()

conn.executescript(sql_script)

In [7]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [8]:
merged_df = orders.merge(
    users,
    on="user_id",
    how="left"
)

In [9]:
final_df = merged_df.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

In [10]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [11]:
final_df.shape

(10000, 12)

In [12]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [13]:
final_df[final_df["membership"]=="Gold"] \
.groupby("city")["total_amount"].sum() \
.sort_values(ascending=False)

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [14]:
final_df.groupby("cuisine")["total_amount"].mean() \
.sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [15]:
user_spend = final_df.groupby("user_id")["total_amount"].sum()
user_spend[user_spend > 1000].count()

np.int64(2544)

In [21]:
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[3.0,3.5,4.0,4.5,5.0]
)

final_df.groupby("rating_range", observed=True)["total_amount"].sum() \
.sort_values(ascending=False)

rating_range
(4.5, 5.0]    2197030.75
(4.0, 4.5]    1960326.26
(3.0, 3.5]    1881754.57
(3.5, 4.0]    1717494.41
Name: total_amount, dtype: float64

In [17]:
final_df[final_df["membership"]=="Gold"] \
.groupby("city")["total_amount"].mean() \
.sort_values(ascending=False)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [18]:
gold_pct = (
    final_df[final_df["membership"]=="Gold"].shape[0]
    / final_df.shape[0]
) * 100

round(gold_pct)

50

In [22]:
final_df["order_date"] = pd.to_datetime(
    final_df["order_date"],
    dayfirst=True
)
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

final_df.groupby("quarter")["total_amount"].sum() \
.sort_values(ascending=False)

quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [20]:
final_df[final_df["rating"] >= 4.5].shape[0]

3374

In [23]:
final_df.shape[0]

10000

In [24]:
final_df[final_df["membership"] == "Gold"].shape[0]

4987

In [25]:
round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)

1889367

In [26]:
final_df["user_id"].nunique()

2883

In [27]:
round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)

np.float64(797.15)

In [28]:
final_df[final_df["rating"] >= 4.5].shape[0]

3374

In [29]:
top_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_city

'Chennai'

In [30]:
final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_city)
].shape[0]

1337

In [31]:
pd.concat(
    [
        final_df.groupby("cuisine")["restaurant_id"].nunique(),
        final_df.groupby("cuisine")["total_amount"].sum()
    ],
    axis=1
).rename(columns={
    "restaurant_id": "restaurant_count",
    "total_amount": "revenue"
}).sort_values("restaurant_count")

,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [37]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

final_df[final_df["restaurant_name_x"].isin(options)] \
.groupby("restaurant_name_x").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
).query("total_orders < 20") \
.sort_values("avg_order_value", ascending=False)

,avg_order_value,total_orders
restaurant_name_x,,
Ruchi Foods Chinese,686.603158,19


In [33]:
final_df.groupby(
    ["membership", "cuisine"]
)["total_amount"].sum().sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [34]:
final_df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_range', 'quarter'],
      dtype='object')